In [ ]:
replace_path = 'data/codex-m/data_sample_random.csv'

loader = CSVLoader(file_path=replace_path)
docs = loader.load()
text_splitter = SemanticChunker(HuggingFaceEmbeddings())
documents = text_splitter.split_documents(docs)
# Instantiate the embedding model
embedder = HuggingFaceEmbeddings()
# Create the vector store 
vector = FAISS.from_documents(documents, embedder)


retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 2})
# Define llm
llm = Ollama(model="mistral")

prompt = """
1. Use the following pieces of context to determine if the final triple present correct fact or not.\n
2. Is the sentence in question correct: answer "1" if it is correct and "0" otherwise.\n
3. If you don't know the answer, just say that "-1"\n
4. Keep the answer limited to the score.\n
5. Start the answer with 'Score:'\n
6. A triple represent a relation between the head entity and the tail entity\n

Context: {context}

Triple: {question}

Helpful Answer:"""


QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt) 

llm_chain = LLMChain(
                llm=llm, 
                prompt=QA_CHAIN_PROMPT, 
                callbacks=None, 
                verbose=False)

document_prompt = PromptTemplate(
    input_variables=["page_content"],
    template="Here a similar triple to help you make a decision:\n:{page_content}, don't forget previous context",
)

combine_documents_chain = StuffDocumentsChain(
                llm_chain=llm_chain,
                document_variable_name="context",
                document_prompt=document_prompt,
                callbacks=None,
            )
qa = RetrievalQA(
                combine_documents_chain=combine_documents_chain,
                verbose=False,
                retriever=retriever,
                return_source_documents=False,
            )